In [ ]:
# Changing the working directory
import os
os.chdir('../')

In [1]:
from utils import *
from datetime import date
from datetime import datetime

In [2]:
student_id = str(6012)

In [3]:
# Getting df of all the students
students = sky.getUsers()
# Filtering for the Student
student = students[students.student_id == student_id]

In [4]:
# Getting the students Enrollments
student_enrollments = sky.get(f'academics/enrollments/{student.id.values[0]}')

In [5]:
# Removing continious courses
student_enrollments = student_enrollments[~student_enrollments.id.isna()]

In [6]:
# Cleaning data to be used in the isActive function
student_enrollments['begin_date'] = pd.to_datetime(student_enrollments['begin_date'])
student_enrollments['end_date'] = pd.to_datetime(student_enrollments['end_date'])

In [7]:
# Adding a column to find active enrollments for students
student_enrollments = sky._isActive(student_enrollments)

In [8]:
# Cleaning the student enrollment data
student_enrollments = student_enrollments[student_enrollments.active == True].reset_index(drop=True)
student_enrollments = student_enrollments[student_enrollments.dropped == 0]

In [9]:
# Getting today's date to look up the day's schedule
today = date.today().strftime("%Y-%m-%d")

In [10]:
# Finding what school level the student is in
student_grade = student['student_info.grade_level'].values[0]
if int(student_grade) in [6, 7, 8]:
    level_abbreviation = 'MS'
elif int(student_grade) in [9, 10, 11, 12]:
    level_abbreviation = 'US'
level_num = sky.get_levels(abbreviation=level_abbreviation, id=True).values[0]

In [11]:
# Getting todays master schedule
master_schedule = sky.get('academics/schedules/master', params={
    'level_num':level_num,
    'start_date':today,
    'end_date':today
})

In [12]:
# Pulling the academic schedule sets 
# Filtering out lunch and other meaningless meeting times
valid_schedule_sets = list(sky.get('academics/schedules/sets', 
             params={'level_num':level_num}
            )['schedule_set_id'].values)

In [13]:
# Getting an array of today's schedule sets
master_schedule_list = master_schedule.schedule_sets.explode()
# Storing active schedule sets in a df
todays_schedule = pd.DataFrame()
for i in range(len(master_schedule_list)):
    # Normalizing the data
    df = pd.json_normalize(master_schedule_list.values[i], 'blocks', 'schedule_set_id')
    # Checking if the schedule set is Academic
    if df['schedule_set_id'].isin(valid_schedule_sets).all():
        todays_schedule = todays_schedule.append(df)

In [14]:
# Filtering applicable schedule sets
if level_abbreviation == "MS":
    # Adding ATH to MS since it's in their daily schedule
    todays_classes = np.where(
        student_enrollments.block_name.isin(todays_schedule.block.tolist()) | 
        student_enrollments.offering_description.isin(['Athletics'])
    )
elif level_abbreviation == "US":
    todays_classes = np.where(
        student_enrollments.block_name.isin(todays_schedule.block.tolist())
    )

In [15]:
students_schedule_today = student_enrollments.iloc[todays_classes]

In [16]:
students_schedule_today

,id,begin_date,block_id,block_name,blocks,building_name,changed_sections,course_code,course_length,course_title,...,level_number,offering_description,offering_id,offering_type_id,room_id,room_name,room_number,school_year,section_identifier,active
1,89077242.0,2022-01-03 00:00:00-05:00,20583.0,P2,True,main,0.0,ENGL0701,2.0,English (7),...,1940,Academics,237148.0,1,34598.0,Room 1208,mai. 1208,2021-2022,1,True
3,89077356.0,2022-01-03 00:00:00-05:00,20585.0,P4,True,main,0.0,SCIE0701,2.0,Life Science (7),...,1940,Academics,237203.0,1,34595.0,Room 1203,mai. 1203,2021-2022,2,True
4,89077300.0,2022-01-03 00:00:00-05:00,20586.0,P5,True,main,0.0,LANG0704,2.0,"Spanish 1A (7,8)",...,1940,Academics,237161.0,1,34625.0,Room 2207,mai. 2207,2021-2022,2,True
5,89077377.0,2022-01-10 11:54:28.033000-05:00,20587.0,P6,True,main,0.0,SCIE0613s,1.0,Programming (8) - Spring,...,1940,Academics,239008.0,1,34873.0,3203,mai. 3203,2021-2022,-2,True
7,89100582.0,2021-10-25 00:00:00-04:00,NaN,,False,,0.0,PHED0705,1.0,Boy's Basketball - Winter,...,1940,Athletics,250333.0,9,NaN,,,2021-2022,,True


In [17]:
todays_schedule

,block_id,block,start_time,end_time,offering_type,is_canceled,schedule_set_id
0,20583,P2,1900-01-01T08:50:00+00:00,1900-01-01T09:45:00+00:00,1,False,10884
1,20585,P4,1900-01-01T09:50:00+00:00,1900-01-01T10:45:00+00:00,1,False,10884
2,20586,P5,1900-01-01T10:55:00+00:00,1900-01-01T11:50:00+00:00,1,False,10884
3,20587,P6,1900-01-01T11:55:00+00:00,1900-01-01T12:50:00+00:00,1,False,10884


In [18]:
sky.get('types/excusedtypes')

,category_description,category_id,excuse_type_id,excuse_description,use_in_calculations,excused,duration,attendance_type
0,Attended Class,1768,5604,Present,True,Present,NA,Present
1,Absent,2183,6702,Absent,True,Unexcused,FullDay,Absence
2,Absent Excused,1770,5606,School Related Absence,True,Excused,NA,Absence
3,Tardy Unexcused,1771,5611,Tardy,True,Unexcused,NA,Tardy
4,Dismissed,2165,6607,Dismissed,True,Unexcused,FullDay,Absence
5,Tardy Excused,1772,5612,Tardy Excused,False,Excused,NA,Tardy
6,Tardy Excused,1772,6900,Bus,False,Excused,NA,Tardy
7,Absent Unexcused,1769,5605,DO NOT USE,True,Unexcused,FullDay,Absence


In [19]:
now = datetime.now().time().strftime("%H:%M:%S")

In [20]:
data={
    'student_user_id': int(student.id.values[0]),
    'begin_date':datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ'),
    'end_date':datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ'),
    'excuse_type_id':5604,
    'excuse_comment':"This was a test - autogenerated by W"
}

In [21]:
attendance_codes = sky.get('types/excusedtypes')

In [22]:
attendance_codes[['category_description', 'excuse_type_id']].rename(columns={
    'category_description':'excuse_name',
    'excuse_type_id':'excuse_id'
}).values.tolist()

[['Attended Class', 5604],
 ['Absent', 6702],
 ['Absent Excused', 5606],
 ['Tardy Unexcused', 5611],
 ['Dismissed', 6607],
 ['Tardy Excused', 5612],
 ['Tardy Excused', 6900],
 ['Absent Unexcused', 5605]]

In [23]:
data

{'student_user_id': 4750432,
 'begin_date': '2022-01-19T13:39:42Z',
 'end_date': '2022-01-19T13:39:42Z',
 'excuse_type_id': 5604,
 'excuse_comment': 'This was a test - autogenerated by W'}

In [24]:
# sky.post(endpoint='attendance', data=data)

In [25]:
datetime.now()

datetime.datetime(2022, 1, 19, 13, 40, 7, 265325)